<a href="https://colab.research.google.com/github/plthiyagu/AI-Engineering/blob/master/07-Machine%20Learning/Association_Mining_Rule_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Association Rule Mining in Retail Store

###Problem Statement:
What are the items that may be frequently purchased together?

###Objective:
To learn how Apriori Algorithm and Association Rules works.

To learn how Combination and Permutation helps to find Support and Confidence of itemsets respectively.

To find frequent itemsets with high confidence and lift, keeping both item together will help to increase sales.

### Introduction
Association rule mining is one of an important technique of data mining for knowledge discovery.

The knowledge of the correlation between the items in the data transaction can use association rule mining.

Retail store analysis is one of an application area of association rule mining technique.

The possible percentage of the correlation of combined items gives the new knowledge. Therefore, it is a very helpful for determiner to take the decisions
Analysis

In [1]:
# Importing Required Library

import pandas as pd
import numpy as np

In [3]:
bread =pd.read_excel('https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/raw_bread.xlsx')

In [4]:
## Here we have transaction data, which include column, Date,Time,Transaction,Item
## we should remove duplicate transaction, it shows quantity of item in same transaction,
## it is not needed in appriori aglo as we only care about different item in particular transaction
bread

,"Date,Time,Transaction,Item"
0,"2016-10-30,09:58:11,1,Bread"
1,"2016-10-30,10:05:34,2,Scandinavian"
2,"2016-10-30,10:05:34,2,Scandinavian"
3,"2016-10-30,10:07:57,3,Hot chocolate"
4,"2016-10-30,10:07:57,3,Jam"
...,...
21288,"2017-04-09,14:32:58,9682,Coffee"
21289,"2017-04-09,14:32:58,9682,Tea"
21290,"2017-04-09,14:57:06,9683,Coffee"
21291,"2017-04-09,14:57:06,9683,Pastry"


In [5]:
## dropping Duplicate Transaction
bread = bread.drop_duplicates()

In [6]:
## we need to split transaction data into Dataframe/tabular structure as follow
new = bread['Date,Time,Transaction,Item'].str.split(',', n = 3, expand = True)

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
## assigning column to data frame "bread"
bread['Date'] = new[0]
bread['Time'] = new[1]
bread['Transaction'] = new[2]
bread['Item'] = new[3]

In [9]:
# in this dataframe we only need column Trasaction and Item, rest is not needed in association mining rule
bread[['Date', 'Time', 'Transaction', 'Item']].head(10)
                                                    

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam
5,2016-10-30,10:07:57,3,Cookies
6,2016-10-30,10:08:41,4,Muffin
7,2016-10-30,10:13:03,5,Coffee
8,2016-10-30,10:13:03,5,Pastry
9,2016-10-30,10:13:03,5,Bread
10,2016-10-30,10:16:55,6,Medialuna


In [10]:
# we need to convert cloumn transacton & item into Crosstab or we can say Binary Matrix as follow
transaction = pd.crosstab(index= bread['Transaction'], columns= bread['Item'])
transaction

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
## Just writing cdv file to check result
## I came to know that, we have one unwanted column named "NONE", we should remove it as follow and proceed further
#tab.to_csv('tab.csv')

In [12]:
## removing unwanted col "NONE"
transaction = transaction.drop(['NONE'], axis = 1)

In [13]:
transaction

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Creating APRIORI_MY function to generate frequent itesets based on minimum threshold support = 0.02

In [14]:
def APRIORI_MY(data, min_support=0.04,  max_length = 4):
    # Collecting Required Library
    import numpy as np
    import pandas as pd
    from itertools import combinations
    # Step 1:
    # Creating a dictionary to stored support of an itemset.
    support = {} 
    L = list(data.columns)
    
    # Step 2: 
    #generating combination of items with len i in ith iteration
    for i in range(1, max_length+1):
        c = set(combinations(L,i))
        
    # Reset "L" for next ith iteration
        L =set()     
    # Step 3: 
        #iterate through each item in "c"
        for j in list(c):
            #print(j)
            sup = data.loc[:,j].product(axis=1).sum()/len(data.index)
            if sup > min_support:
                #print(sup, j)
                support[j] = sup
                
                # Appending frequent itemset in list "L", already reset list "L" 
                L = list(set(L) | set(j))
        
    # Step 4: data frame with cols "items", 'support'
    result = pd.DataFrame(list(support.items()), columns = ["Items", "Support"])
    return(result)

In [15]:
## finding frequent itemset with min support = 4%
my_freq_itemset = APRIORI_MY(transaction, 0.04, 3)
my_freq_itemset.sort_values(by = 'Support', ascending = False)

,Items,Support
5,"(Coffee,)",0.475081
7,"(Bread,)",0.324940
1,"(Tea,)",0.141643
4,"(Cake,)",0.103137
9,"(Bread, Coffee)",0.089393
2,"(Pastry,)",0.085510
6,"(Sandwich,)",0.071346
8,"(Medialuna,)",0.061379
3,"(Hot chocolate,)",0.057916
11,"(Cake, Coffee)",0.054349


Creating ASSOCIATION_RULE_MY function to generate itemset based on minimun threshold confidence.


In [16]:
def ASSOCIATION_RULE_MY(df, min_threshold=0.5):
    import pandas as pd
    from itertools import permutations
    
    # STEP 1:
    #creating required varaible
    support = pd.Series(df.Support.values, index=df.Items).to_dict()
    data = []
    L= df.Items.values
    
    # Step 2:
    #generating rule using permutation
    p = list(permutations(L, 2))
    
    # Iterating through each rule
    for i in p:
        
        # If LHS(Antecedent) of rule is subset of RHS then valid rule.
        if set(i[0]).issubset(i[1]):
            conf = support[i[1]]/support[i[0]]
            #print(i, conf)
            if conf > min_threshold:
                #print(i, conf)
                j = i[1][not i[1].index(i[0][0])]
                lift = support[i[1]]/(support[i[0]]* support[(j,)])
                leverage = support[i[1]] - (support[i[0]]* support[(j,)])
                convection = (1 - support[(j,)])/(1- conf)
                data.append([i[0], (j,), support[i[0]], support[(j,)], support[i[1]], conf, lift, leverage, convection])

         
    # STEP 3:
    result = pd.DataFrame(data, columns = ["antecedents", "consequents", "antecedent support", "consequent support",
                                        "support", "confidence", "Lift", "Leverage", "Convection"])
    return(result)

In [17]:
## Rule with minimun confidence = 50%
my_rule = ASSOCIATION_RULE_MY(my_freq_itemset, 0.5)
my_rule

,antecedents,consequents,antecedent support,consequent support,support,confidence,Lift,Leverage,Convection
0,"(Pastry,)","(Coffee,)",0.085510,0.475081,0.047214,0.552147,1.162216,0.00659,1.172079
1,"(Cake,)","(Coffee,)",0.103137,0.475081,0.054349,0.526958,1.109196,0.00535,1.109667


Finally sorting results by Lift to get highly associated itemsets.

In [18]:
my_rule.sort_values(by='Lift', ascending= False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,Lift,Leverage,Convection
0,"(Pastry,)","(Coffee,)",0.085510,0.475081,0.047214,0.552147,1.162216,0.00659,1.172079
1,"(Cake,)","(Coffee,)",0.103137,0.475081,0.054349,0.526958,1.109196,0.00535,1.109667


Cross Verifying results with apriori and association rule from mlxtend -> frequent patterns package

In [19]:
# Loading standard package
from mlxtend.frequent_patterns import apriori, association_rules

In [20]:
## finding frequent itemset with min support = 4%
frequent_itemset = apriori(df = transaction, min_support= 0.04, use_colnames= True)
frequent_itemset.sort_values(by = 'support', ascending = False)

,support,itemsets
2,0.475081,(Coffee)
0,0.324940,(Bread)
8,0.141643,(Tea)
1,0.103137,(Cake)
9,0.089393,"(Coffee, Bread)"
6,0.085510,(Pastry)
7,0.071346,(Sandwich)
5,0.061379,(Medialuna)
4,0.057916,(Hot chocolate)
10,0.054349,"(Coffee, Cake)"


Createing associate rule such that item brought with conditional probability(Confidence) more than 50% with corresponding item

In [21]:
## Rule with minimun confidence = 50%
Rules = association_rules(frequent_itemset, min_threshold= 0.5)
Rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cake),(Coffee),0.103137,0.475081,0.054349,0.526958,1.109196,0.00535,1.109667
1,(Pastry),(Coffee),0.085510,0.475081,0.047214,0.552147,1.162216,0.00659,1.172079


In [22]:
## Finally sorting results by Lift to get highly associated itemsets.
Rules.sort_values(by='lift', ascending= False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(Pastry),(Coffee),0.085510,0.475081,0.047214,0.552147,1.162216,0.00659,1.172079
0,(Cake),(Coffee),0.103137,0.475081,0.054349,0.526958,1.109196,0.00535,1.109667


Conclusion

Results from developed function(APRIORI_MY, ASSOCIATION_RULE_MY) has matched with builts packages.

it is observed that "Toast" & "Coffee" are highly associated with lift 1.48.
Coffee has been brought most frequently with 47.5% of all the transcaction